In [1]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
from pyspark.sql import SparkSession



In [8]:
spark = SparkSession.builder.getOrCreate()

In [9]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
],  schema = 'a long, b double, c string, d date, e timestamp'

)
print(df)

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]


In [10]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [12]:
df.show(0)

+---+---+---+---+---+
|  a|  b|  c|  d|  e|
+---+---+---+---+---+
+---+---+---+---+---+
only showing top 0 rows



In [14]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
4,5.0,string3,2000-03-01,2000-01-03 12:00:00


In [15]:
df.show(1, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
only showing top 1 row



In [18]:
df.columns
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [19]:
df.select("a","b","c").describe().show()

+-------+------------------+------------------+-------+
|summary|                 a|                 b|      c|
+-------+------------------+------------------+-------+
|  count|                 3|                 3|      3|
|   mean|2.3333333333333335|3.3333333333333335|   null|
| stddev|1.5275252316519468|1.5275252316519468|   null|
|    min|                 1|               2.0|string1|
|    max|                 4|               5.0|string3|
+-------+------------------+------------------+-------+



In [21]:
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
4,5.0,string3,2000-03-01,2000-01-03 12:00:00


In [24]:
df.filter(df.a ==1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [25]:
df_gro = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df_gro.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



# First real live project 
ETL Extract Transform Load

In [4]:
import requests
import pandas as pd
from pyspark.sql import SparkSession

def get_dataset_from_github(url):
    """This function allows you to retrieve data from GitHub repositories.
    
    Input:
    - url: String from the GitHub repository for the dataset in raw data
    
    Output:
    - A dataset if successful, otherwise None
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an exception if the status code is not 200 (OK)
        dataset = pd.read_csv(url)
        return dataset
    except requests.exceptions.RequestException as e:
        print('Error downloading the dataset:', e)
        return None
#####
    
# Initialize Spark session
spark = SparkSession.builder.appName("GitHub Dataset").getOrCreate()

# Get the dataset from GitHub and convert it into a pandas DataFrame
df_pd = get_dataset_from_github(url='https://raw.githubusercontent.com/jhnwr/auto-reporting/main/report1.csv')

# Check if the DataFrame is not None
if df_pd is not None:
    # Convert pandas DataFrame to Spark DataFrame
    df_spark = spark.createDataFrame(df_pd)
    df_spark.show()
else:
    print("Dataset could not be retrieved.")


+--------------------+----------+--------------------+----------+----------+--------------------+-------------+-------+--------+---+
|            order_id|      date|               email|first_name| last_name|             address|      country|   item|    size|qty|
+--------------------+----------+--------------------+----------+----------+--------------------+-------------+-------+--------+---+
|01HPYDWKXGEBJD9ZJ...| 7/19/2023|  brieger0@skype.com|       Bat|    Rieger|8529 Park Meadow ...|United States|  Hoody|XX-Large|  2|
|01HPYDWKZDZCTC2TP...| 7/10/2023|   mabbitt1@bing.com|      Mary|    Abbitt|     8 Debs Junction|       Norway| Gloves|  Medium|  1|
|01HPYDWKZGM48TRP1...| 10/7/2023|ckeave2@bravesite...|   Cordula|     Keave|  618 Packers Avenue|       Brazil|Sweater| X-Large|  2|
|01HPYDWKZJ71D89EN...| 9/27/2023| rgymlett3@bbc.co.uk|     Rhoda|   Gymlett|       30 Stang Lane|       Sweden|  Hoody|XX-Large|  1|
|01HPYDWKZNDT89QSV...|  2/8/2023|   mdorre4@google.cn|      Mead|    